<a href="https://colab.research.google.com/github/nvhmadridista/BrainTumorDiagnosisUsingMRI/blob/feature%2Fmodel-training/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
PROJECT_DIR = '/content/drive/MyDrive/ML'
MODELS_DIR = os.path.join(PROJECT_DIR, 'models')
LOGS_DIR = os.path.join(PROJECT_DIR, 'logs')
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
print("Đã gắn Drive và thiết lập thư mục lưu trữ.")

Mounted at /content/drive
Đã gắn Drive và thiết lập thư mục lưu trữ.


In [2]:

!git clone -b feature/model-training https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI.git ML
%cd ML
!git branch

Cloning into 'ML'...
remote: Enumerating objects: 12475, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 12475 (delta 31), reused 101 (delta 20), pack-reused 12349 (from 1)
Receiving objects: 100% (12475/12475), 206.87 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/ML
* feature/model-training


In [3]:
REPO_URL = f'https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI/tree/feature/model-training'
REPO_NAME = 'ML'
!git clone {REPO_URL}
%cd {REPO_NAME}


Cloning into 'model-training'...
fatal: repository 'https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI/tree/feature/model-training/' not found
[Errno 2] No such file or directory: 'ML'
/content/ML


In [4]:

%cd /content/ML
DRIVE_ZIP_PATH = '/content/drive/MyDrive/KaggleData/archive.zip'
DATASET_DIR = 'dataset'
os.makedirs(DATASET_DIR, exist_ok=True)
!unzip -q {DRIVE_ZIP_PATH} -d {DATASET_DIR}
print("Đã giải nén dữ liệu gốc vào thư mục: /content/ML/dataset")

/content/ML
Đã giải nén dữ liệu gốc vào thư mục: /content/ML/dataset


In [5]:
from pathlib import Path
import math, shutil
import os
# Resplit images in dataset
BASE = Path("dataset")
SRC_TRAIN = BASE / "Training"
SRC_TEST = BASE / "Testing"
OUT_BASE = Path("data_split")
OUT_TRAIN = OUT_BASE / "train"
OUT_VAL = OUT_BASE / "val"
OUT_TEST = OUT_BASE / "test"

OUT_BASE.mkdir(parents=True, exist_ok=True)

classes = sorted({p.name for p in SRC_TRAIN.iterdir() if p.is_dir()} | {p.name for p in SRC_TEST.iterdir() if p.is_dir()})
for c in classes:
    files = []
    ct = SRC_TRAIN / c
    if ct.exists():
        files += sorted(ct.rglob("*.jpg"))
    cte = SRC_TEST / c
    if cte.exists():
        files += sorted(cte.rglob("*.jpg"))
    n = len(files)
    n_train = math.floor(n * 0.70)
    n_val = math.floor(n * 0.15)
    n_test = n - n_train - n_val
    splits = [(OUT_TRAIN / c, files[:n_train]), (OUT_VAL / c, files[n_train:n_train+n_val]), (OUT_TEST / c, files[n_train+n_val:])]
    for out_dir, group in splits:
        out_dir.mkdir(parents=True, exist_ok=True)
        for f in group:
            dst = out_dir / f.name
            if f.resolve() == dst.resolve():
                continue
            shutil.move(str(f), str(dst))

In [6]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src/preprocessing')))
from run_preprocessing import run_main_preprocess
if __name__ == "__main__":
    tasks = [
        ("data_split/train", "data/processed/train"),
        ("data_split/val", "data/processed/val"),
        ("data_split/test", "data/processed/test"),
    ]
os.makedirs("data/processed", exist_ok=True)

for raw_dir, out_dir in tasks:
    print(f"\n RUNNING PREPROCESSING: {raw_dir} - {out_dir} ")
    run_main_preprocess(raw_dir, out_dir)

print("\n All Preprocessing complete. Data ready for Training.")


 RUNNING PREPROCESSING: data_split/train - data/processed/train 
Preprocessing class: notumor


100%|██████████| 1400/1400 [00:12<00:00, 107.79it/s]


Preprocessing class: meningioma


100%|██████████| 1151/1151 [00:10<00:00, 109.47it/s]


Preprocessing class: glioma


...

Preprocessing class: glioma


100%|██████████| 244/244 [00:03<00:00, 78.71it/s]


Preprocessing class: pituitary


100%|██████████| 265/265 [00:02<00:00, 98.75it/s] 


 DONE! Preprocessed images from data_split/test saved in data/processed/test

 All Preprocessing complete. Data ready for Training.


In [7]:
!python -m src.training.train

Device: cuda
Đã lưu tên lớp: /content/drive/MyDrive/ML/models/class_names.json

Epoch 1/25
Train Loss: 7.5234 | Train Acc: 0.3274
Val Loss:   2.4099 | Val Acc:   0.3774
Saved best model! Val Loss: 2.4099

Epoch 2/25
Train Loss: 1.7874 | Train Acc: 0.4298
Val Loss:   1.6850 | Val Acc:   0.4154
Saved best model! Val Loss: 1.6850

Epoch 3/25
Train Loss: 1.3923 | Train Acc: 0.5330
Val Loss:   1.6135 | Val Acc:   0.5390
Saved best model! Val Loss: 1.6135

Epoch 4/25
Train Loss: 0.9221 | Train Acc: 0.7120
Val Loss:   1.3166 | Val Acc:   0.6445
Saved best model! Val Loss: 1.3166

Epoch 5/25
Train Loss: 0.5044 | Train Acc: 0.8480
Val Loss:   1.0923 | Val Acc:   0.6340
Saved best model! Val Loss: 1.0923

Epoch 6/25
Train Loss: 0.3450 | Train Acc: 0.8993
Val Loss:   1.1862 | Val Acc:   0.7728

Epoch 7/25
Train Loss: 0.2248 | Train Acc: 0.9361
Val Loss:   0.8442 | Val Acc:   0.7937
Saved best model! Val Loss: 0.8442

Epoch 8/25
Train Loss: 0.0899 | Train Acc: 0.9699
Val Loss:   1.7015 | Val Acc: 

In [8]:
%cd /content/ML
ZIP_FILENAME = "brain_data_raw_and_split.zip"
!zip -r -q {ZIP_FILENAME}  data_split/ data/processed/
print(f"Đã tạo file nén chứa tất cả dữ liệu: {ZIP_FILENAME}")


/content/ML
Đã tạo file nén chứa tất cả dữ liệu: brain_data_raw_and_split.zip


In [9]:
DRIVE_DATA_BACKUP = '/content/drive/MyDrive/ML/Data_Backup'
!mkdir -p {DRIVE_DATA_BACKUP}
!cp {ZIP_FILENAME} {DRIVE_DATA_BACKUP}/
print(f"Đã sao chép file nén lên Drive thành công tại: {DRIVE_DATA_BACKUP}/{ZIP_FILENAME}")

Đã sao chép file nén lên Drive thành công tại: /content/drive/MyDrive/ML/Data_Backup/brain_data_raw_and_split.zip
